In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn import feature_extraction, linear_model, model_selection, metrics
from sklearn import ensemble
from scipy import sparse
import sys
sys.path.append("..")
from src.make_model_lstm import hate_speech_model

/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
df = pd.read_csv('../data/labeled_data.csv')

In [4]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [5]:
df[['hate_speech', 'offensive_language', 'neither']].head()

,hate_speech,offensive_language,neither
0,0,0,3
1,0,3,0
2,0,3,0
3,0,2,1
4,0,6,0


In [6]:
stop_words = set(stopwords.words('english')) 

In [7]:
token = nltk.tokenize.casual.TweetTokenizer()

In [34]:
tfidf = feature_extraction.text.TfidfVectorizer(strip_accents = 'ascii',
                                                lowercase = True,
                                                tokenizer = token.tokenize,
                                                stop_words = stop_words,
                                                ngram_range = (1, 3),
                                                max_features = 100000

)

In [35]:
tweets = df['tweet'].apply(lambda x: x.lstrip('!'))

In [36]:
X = tfidf.fit_transform(tweets)

In [37]:
y = np.argmax(df[['hate_speech', 'offensive_language', 'neither']].values, axis = 1)

In [38]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = .1)

In [39]:
y_cat = np.zeros((len(y_test), 3))
for i, l in enumerate(y_test):
    y_cat[i, l] = 1

In [40]:
counts = {}
for val in y:
    if val in counts:
        counts[val] += 1
    else:
        counts[val] = 1
        
print(counts)

{2: 4163, 1: 19190, 0: 1430}


In [41]:
num0 = len(y_train[y_train == 0])
num1 = len(y_train[y_train == 1])
num2 = len(y_train[y_train == 2])

y_train0 = y_train[y_train == 0]
X_train0 = X_train[y_train == 0]

ind1 = np.random.choice(num1, size = (num0*2, ))
ind2 = np.random.choice(num2, size = (num0*2, ))
y_train1 = y_train[ind1]
X_train1 = X_train[ind1]

y_train2 = y_train[ind2]
X_train2 = X_train[ind2]

X_train_down = sparse.vstack([X_train0, X_train1, X_train2])
y_train_down = np.hstack([y_train0, y_train1, y_train2])



In [42]:
X_train_down.shape

(6375, 100000)

In [43]:
LR = linear_model.LogisticRegression()
LR.fit(X_train_down, y_train_down)

LogisticRegression()

[2 1 2 ... 1 2 1]


In [27]:
probs = LR.predict_proba(X_test)
preds = LR.predict(X_test)
print(metrics.roc_auc_score(y_cat, probs))
LR.score(X_test, y_test)

0.9128966870741534


0.8200887454618798

In [20]:
"""model = ensemble.GradientBoostingClassifier()
model.fit(X_train, y_train)
probs = model.predict_proba(X_test)
print(metrics.roc_auc_score(y_cat, probs))
model.score(X_test, y_test)""";

In [44]:
class_split = (y_test == 2)

Xc = X_test[class_split]
Xnc = X_test[~class_split]
yc = y_test[class_split]
ync = y_test[~class_split]

In [45]:
preds = LR.predict(Xc)
preds2 = LR.predict(Xnc)
print(metrics.accuracy_score(yc, preds))
print(metrics.accuracy_score(ync, preds2))


0.2748267898383372
0.916911045943304


In [39]:
vocab =  tfidf.vocabulary_

In [50]:
X_train_down

<6390x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 79376 stored elements in Compressed Sparse Row format>

In [41]:
tokenized = np.array(tweets.apply(token.tokenize))

In [151]:
maxWords = max(map(len, tokenized))
avgword = sum(map(len, tokenized))/ len(list(map(len, tokenized)))

In [153]:
avgword, maxWords

(16.317919541621272, 53)

In [47]:
def create_seq(sent, vocab, maxWords):
    n = len(sent)
    numZeros = maxWords - n
    result = [0]*numZeros
    
    for word in sent:
        if word in vocab:
            result.append(vocab[word] + 2)
        else:
            result.append(1)
            
    return result

In [53]:
sequences = np.array(list(map(lambda x: create_seq(x, vocab, maxWords), tokenized)))

In [54]:
sequences

array([[    0,     0,     0, ..., 17697,     1,  1821],
       [    0,     0,     0, ..., 13874,     2,     2],
       [    0,     0,     0, ...,  7517,     1, 16032],
       ...,
       [    0,     0,     0, ...,  8090,     1,     1],
       [    0,     0,     0, ..., 17321,     1, 11922],
       [    0,     0,     0, ...,     1,     1,     1]])

In [57]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(sequences, y_cat, test_size = .1)

In [72]:
print(y_train2)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [106]:
num0 = len(y_train[y_train[:, 0] == 1])
num1 = len(y_train[y_train[:, 1] == 1])
num2 = len(y_train[y_train[:, 2] == 1])

y_train0 = y_train[y_train[:, 0] == 1]
X_train0 = X_train[y_train[:, 0] == 1]

ind1 = np.random.choice(num1, size = (num0, ))
ind2 = np.random.choice(num2, size = (num0, ))
y_train1 = y_train[ind1]
X_train1 = X_train[ind1]

y_train2 = y_train[ind2]
X_train2 = X_train[ind2]

X_train_down = np.vstack([X_train0, X_train1, X_train2])
y_train_down = np.vstack([y_train0, y_train1, y_train2])

In [117]:
print(X_train0.shape, X_train1.shape, X_train2.shape)
print((X_train_down).shape)

(1283, 53) (1283, 53) (1283, 53)
(3849, 53)


>> Compiled...
Train on 3656 samples, validate on 193 samples
Epoch 1/14
3656/3656 [==============================] - 14s 4ms/sample - loss: 0.9957 - val_loss: 1.0384
Epoch 2/14
3656/3656 [==============================] - 11s 3ms/sample - loss: 0.9388 - val_loss: 1.0163
Epoch 3/14
3656/3656 [==============================] - 10s 3ms/sample - loss: 0.8692 - val_loss: 0.7051
Epoch 4/14
3656/3656 [==============================] - 11s 3ms/sample - loss: 0.6207 - val_loss: 0.7734
Epoch 5/14
3656/3656 [==============================] - 11s 3ms/sample - loss: 0.4180 - val_loss: 0.7520
Epoch 6/14
3656/3656 [==============================] - 11s 3ms/sample - loss: 0.2956 - val_loss: 0.6409
Epoch 7/14
3656/3656 [==============================] - 11s 3ms/sample - loss: 0.2282 - val_loss: 0.5213
Epoch 8/14
3656/3656 [==============================] - 11s 3ms/sample - loss: 0.1815 - val_loss: 0.5918
Epoch 9/14
3656/3656 [==============================] - 10s 3ms/sample - loss: 0.1449 - val_loss: 

In [145]:
print(metrics.roc_auc_score(y_test, preds))

0.8070023060663809


In [146]:
y_true = np.argmax(y_test, axis = 1)
y_pred = np.argmax(preds, axis = 1)

In [147]:
print(metrics.classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.17      0.59      0.26       147
           1       0.89      0.78      0.83      1922
           2       0.71      0.47      0.57       410

    accuracy                           0.72      2479
   macro avg       0.59      0.61      0.55      2479
weighted avg       0.81      0.72      0.75      2479



In [148]:
print(metrics.confusion_matrix(y_true, y_pred))

[[  87   51    9]
 [ 356 1494   72]
 [  78  138  194]]
